In [ ]:
shh <- suppressPackageStartupMessages # It's a library, so shhh!
options(warn=-2)
shh(library(data.table))
shh(library(dplyr))

In [ ]:
fread_evoef1 <- function(x){
    out <- fread(cmd = paste(" grep '=' ", x , " | ", "grep 'Total' " ), header = F, col.names = c("Term","A","EVOEF1")) %>%
        select(EVOEF1)
    splitd <- strsplit(basename(x), split="_", fixed = T)[[1]]
    out$label <- splitd[[1]]
    out$frame <- splitd[[2]]
    return(out)
}

In [ ]:
rezevoef = bind_rows(lapply(X = snakemake@input$promod_models_evoef1_evals, fread_evoef1))


In [ ]:
fread_prodigy <- function(x){
    out <- fread(x) %>%
        setNames(paste0('PRODIGY_', names(.)))
    splitd <- strsplit(basename(x), split="_", fixed = T)[[1]] 
    out$COMPLEXID <- NULL
    out$label <- splitd[[1]]
    out$frame <- splitd[[2]]
    return(out)
}

In [ ]:
rezprodigy = bind_rows(lapply(X = snakemake@input$promod_models_prodigy_evals, fread_prodigy))


In [ ]:
out_wide_full <- rezprodigy %>%
    left_join(rezevoef, by=c("label","frame"))
out_wide_full
    

In [ ]:
out_wide_main <- out_wide_full %>%
    select(label, frame, EVOEF1, PRODIGY=PRODIGY_Predicted_binding_affinity)
out_wide_main_static <- out_wide_main %>%
    filter(frame == "1")
out_wide_main_dynamic_MEAN <- out_wide_main %>%
    filter(frame != "1") %>%
    group_by(label) %>%
    summarise_if(is.numeric, mean, na.rm = TRUE) %>%
    rename_if(is.numeric, .funs = ~ paste0("MEAN_", .))
out_wide_main_dynamic_MEDIAN <- out_wide_main %>%
    filter(frame != "1") %>%
    group_by(label) %>%
    summarise_if(is.numeric, median, na.rm = TRUE) %>%
    rename_if(is.numeric, .funs = ~ paste0("MEDIAN_", .))
out_wide_main <- out_wide_main_static %>%
    left_join(out_wide_main_dynamic_MEAN, by = "label") %>%
    left_join(out_wide_main_dynamic_MEDIAN, by = "label")


In [ ]:
#calculate dEVOEF1,dPRODIGY
outdf <- out_wide_main %>%
    select(-starts_with("MEDIAN"), -starts_with("MEAN"), -frame) %>%
    rowwise() %>%
    mutate(stem = paste(strsplit(label,split = "=", fixed = T)[[1]][1:2], collapse = "=") ) # to match wt with mutants
outdf4ref <- outdf %>%
    filter(grepl(pattern = "=nan",fixed = T, x = label)) %>%
    mutate(EVOEF1_ref = EVOEF1, PRODIGY_ref = PRODIGY) %>%
    select(-label, -EVOEF1, -PRODIGY)
    
outdf <- outdf %>%
    left_join(outdf4ref, by = "stem") %>%
    select(-stem) %>%
    mutate(dEVOEF1 = EVOEF1 - EVOEF1_ref, dPRODIGY = PRODIGY -PRODIGY_ref) %>%
    select(label,dEVOEF1,dPRODIGY) %>%
    filter(!grepl(pattern = "=nan", fixed = T, x = label))
outdf

In [ ]:
ddgdf <- fread(snakemake@input$ddg) 

outdf2 <- outdf %>% 
    left_join(ddgdf, by="label")

In [ ]:
fwrite(file = snakemake@output$ddg_results_on_promod_full, x = out_wide_full) 
fwrite(file = snakemake@output$ddg_results_on_promod_main, x = outdf2) 